In [1]:
import evaluate
import functools

Write here just incase i forget


evaluate's sacrebleu model looks at len 5 array prediction as a single tokenized prediction


this will cause problems in EvalBase.py
specifically model eval function. instead of generating 5 individual scores, it will only generate one
this will cause the program to break down on batched_corr function because it assumes it generate 5, and compares to 5 human scores

cc = eval(f"scipy.stats.{corr_metric}")(human_score, metric_score)[0]

scipy metic will break cuz human score len = 5
but metric score len = 1

In [2]:
model = evaluate.load("sacrebleu")

In [3]:
predictions = ["With plenty at stake for both United and City -- both sides are challenging for a top-four spot in the Premier League -- some may feel that the 4pm start could be unwise as it allows supporters more time in the pub .    Police say they have no objections to the late afternoon kick-off for Sunday 's Manchester derby at Old Trafford .    However , the man in charge of patrolling the sell-out clash has no such fears .",
 "police say they have no objections to the late afternoon kick-off for sunday 's manchester derby at old trafford .   police say they have no objections to sunday 's manchester derby taking place in the late afternoon .   rio ferdinand was hit by an object thrown from the crowd during the manchester derby in december 2012 .",
 "Police say they have no objections to the late afternoon kick-off for Sunday 's Manchester derby at Old Trafford .   With plenty at stake for both United and City -- both sides are challenging for a top-four spot in the Premier League -- some may feel that the 4pm start could be unwise as it allows supporters more time in the pub .   However , the man in charge of patrolling the sell-out clash has no such fears .   Chief Superintendent John O'Hare says the kick-off was agreed by all parties and revealed that the decision is down to good behaviour from supporters of each side .",
 "police say they have no objections to the late afternoon kick-off for sunday 's manchester derby at old trafford .   police say they have no objections to sunday 's manchester derby taking place in the late afternoon   however , the man in charge of patrolling the sell-out clash has no such fears .",
 "police say they have no objections to the late afternoon kick-off for sunday 's manchester derby at old trafford .   police say they have no objections to sunday 's manchester derby taking place in the late afternoon .   however , the man in charge of patrolling the sell-out clash has no such fears ."]

In [4]:
references = ["manchester united take on manchester city on sunday .    match will begin at 4pm local time at united 's old trafford home .    police have no objections to kick-off being so late in the afternoon .    last late afternoon weekend kick-off in the manchester derby saw 34 fans arrested at wembley in 2011 fa cup semi-final .",
 "manchester united take on manchester city on sunday .    match will begin at 4pm local time at united 's old trafford home .    police have no objections to kick-off being so late in the afternoon .    last late afternoon weekend kick-off in the manchester derby saw 34 fans arrested at wembley in 2011 fa cup semi-final .",
 "manchester united take on manchester city on sunday .    match will begin at 4pm local time at united 's old trafford home .    police have no objections to kick-off being so late in the afternoon .    last late afternoon weekend kick-off in the manchester derby saw 34 fans arrested at wembley in 2011 fa cup semi-final .",
 "manchester united take on manchester city on sunday .    match will begin at 4pm local time at united 's old trafford home .    police have no objections to kick-off being so late in the afternoon .    last late afternoon weekend kick-off in the manchester derby saw 34 fans arrested at wembley in 2011 fa cup semi-final .",
 "manchester united take on manchester city on sunday .    match will begin at 4pm local time at united 's old trafford home .    police have no objections to kick-off being so late in the afternoon .    last late afternoon weekend kick-off in the manchester derby saw 34 fans arrested at wembley in 2011 fa cup semi-final ."]

In [5]:
len(predictions)

5

In [6]:
len(references)

5

### indicated below will only generate 1 score

In [7]:
results = model.compute(predictions=predictions, references=references)

In [8]:
results

{'score': 6.563580991514543,
 'counts': [119, 39, 11, 5],
 'totals': [350, 345, 340, 335],
 'precisions': [34.0,
  11.304347826086957,
  3.235294117647059,
  1.492537313432836],
 'bp': 1.0,
 'sys_len': 350,
 'ref_len': 285}

In [9]:
model = functools.partial(evaluate.load("sacrebleu").compute,  use_aggregator=False)

In [10]:
results = model(predictions=predictions, references=references)

TypeError: _compute() got an unexpected keyword argument 'use_aggregator'

### Rouge doesnt have this problem because the aggregator can be turned off

In [11]:
rouge = functools.partial(evaluate.load("rouge").compute,  use_aggregator=False)

In [12]:
results = rouge(predictions=predictions, references=references)

In [13]:
results

{'rouge1': [0.3939393939393939,
  0.4324324324324324,
  0.3478260869565218,
  0.42592592592592593,
  0.42592592592592593],
 'rouge2': [0.13846153846153847,
  0.16513761467889906,
  0.12578616352201258,
  0.1509433962264151,
  0.1509433962264151],
 'rougeL': [0.2727272727272727,
  0.3063063063063063,
  0.1739130434782609,
  0.27777777777777773,
  0.27777777777777773],
 'rougeLsum': [0.2727272727272727,
  0.3063063063063063,
  0.1739130434782609,
  0.27777777777777773,
  0.27777777777777773]}

### Bleu doesnt have a option to turn off the aggregator

### to fix the problem, need to add 1 more loop to generate 5

In [16]:
model = evaluate.load("sacrebleu")
for p,r in zip(predictions, references):
    results = model.compute(predictions=[p], references=[r])
    print(results)

{'score': 5.141544895026856, 'counts': [21, 6, 2, 1], 'totals': [79, 78, 77, 76], 'precisions': [26.582278481012658, 7.6923076923076925, 2.5974025974025974, 1.3157894736842106], 'bp': 1.0, 'sys_len': 79, 'ref_len': 57}
{'score': 9.52398752064215, 'counts': [26, 10, 3, 1], 'totals': [57, 56, 55, 54], 'precisions': [45.6140350877193, 17.857142857142858, 5.454545454545454, 1.8518518518518519], 'bp': 1.0, 'sys_len': 57, 'ref_len': 57}
{'score': 3.8636626550648483, 'counts': [23, 6, 2, 1], 'totals': [107, 106, 105, 104], 'precisions': [21.49532710280374, 5.660377358490566, 1.9047619047619047, 0.9615384615384616], 'bp': 1.0, 'sys_len': 107, 'ref_len': 57}
{'score': 7.972619098465378, 'counts': [24, 8, 2, 1], 'totals': [53, 52, 51, 50], 'precisions': [45.283018867924525, 15.384615384615385, 3.9215686274509802, 2.0], 'bp': 0.9273059746713349, 'sys_len': 53, 'ref_len': 57}
{'score': 8.300702902525652, 'counts': [25, 9, 2, 1], 'totals': [54, 53, 52, 51], 'precisions': [46.2962962962963, 16.98113

### This is a good start but it also cannot fit the function because these values are in serperated dictionaries, a good result shuld look like the one below

In [17]:
{'rouge1': [0.3939393939393939, 0.4324324324324324, 0.3478260869565218, 0.42592592592592593, 0.42592592592592593, 0.4324324324324324, 0.3934426229508197, 0.3934426229508197, 0.38095238095238093, 0.42592592592592593, 0.49645390070921985], 'rouge2': [0.13846153846153847, 0.16513761467889906, 0.12578616352201258, 0.1509433962264151, 0.1509433962264151, 0.16513761467889906, 0.15000000000000002, 0.15000000000000002, 0.15533980582524268, 0.1509433962264151, 0.18705035971223022], 'rougeL': [0.2727272727272727, 0.3063063063063063, 0.1739130434782609, 0.27777777777777773, 0.27777777777777773, 0.3063063063063063, 0.26229508196721313, 0.22950819672131145, 0.24761904761904763, 0.27777777777777773, 0.25531914893617025], 'rougeLsum': [0.2727272727272727, 0.3063063063063063, 0.1739130434782609, 0.27777777777777773, 0.27777777777777773, 0.3063063063063063, 0.26229508196721313, 0.22950819672131145, 0.24761904761904763, 0.27777777777777773, 0.25531914893617025]}

{'rouge1': [0.3939393939393939,
  0.4324324324324324,
  0.3478260869565218,
  0.42592592592592593,
  0.42592592592592593,
  0.4324324324324324,
  0.3934426229508197,
  0.3934426229508197,
  0.38095238095238093,
  0.42592592592592593,
  0.49645390070921985],
 'rouge2': [0.13846153846153847,
  0.16513761467889906,
  0.12578616352201258,
  0.1509433962264151,
  0.1509433962264151,
  0.16513761467889906,
  0.15000000000000002,
  0.15000000000000002,
  0.15533980582524268,
  0.1509433962264151,
  0.18705035971223022],
 'rougeL': [0.2727272727272727,
  0.3063063063063063,
  0.1739130434782609,
  0.27777777777777773,
  0.27777777777777773,
  0.3063063063063063,
  0.26229508196721313,
  0.22950819672131145,
  0.24761904761904763,
  0.27777777777777773,
  0.25531914893617025],
 'rougeLsum': [0.2727272727272727,
  0.3063063063063063,
  0.1739130434782609,
  0.27777777777777773,
  0.27777777777777773,
  0.3063063063063063,
  0.26229508196721313,
  0.22950819672131145,
  0.24761904761904763,
  0.2

### custom code a bleu function

In [18]:
import evaluate
class custom_blue():
    
    def __init__(self):
        self.model = evaluate.load("sacrebleu")
    
    #input is [str,str,str]
    #hard coded the score
    def compute(self, predictions, references):
        result = {'score':[]}
        for p,r in zip(predictions, references):
            result['score'].append(self.model.compute(predictions=[p], references=[r])['score'])
        return result

In [19]:
custom_blue().compute(predictions, references)

{'score': [5.141544895026856,
  9.52398752064215,
  3.8636626550648483,
  7.972619098465378,
  8.300702902525652]}